# periocular dataset generator

In [2]:
import os
import glob
import cv2
import numpy as np
import face_alignment
from skimage import io

# gpu버전
# fa = face_alignment.FaceAlignment(face_alignment.LandmarksType._2D, flip_input=False)
# cpu버전
fa = face_alignment.FaceAlignment(face_alignment.LandmarksType._2D, device='cpu')
n = 0
PRLAB_path='./mask dataset' 
PRLAB_folder_list= os.listdir(PRLAB_path) #PRLAB 내부 폴더 리스트
#print (PRLAB_folder_list)

for folder_num in range(0, len(PRLAB_folder_list)):
    #print('PRLAB_folder: %s' %PRLAB_folder_list[folder_num])
    dirs = PRLAB_path + '/' + PRLAB_folder_list[folder_num]
#     print('dirs: %s' %dirs)
    filelist = os.listdir(dirs)
    
    for img_num in range(0,len(filelist)):
        file_path = dirs + '/' + filelist[img_num]
        stream = open(file_path.encode("utf-8"), "rb")
        bytes = bytearray(stream.read())
        numpyArray = np.asarray(bytes, dtype=np.uint8)
        try:
            img = cv2.imdecode(numpyArray,cv2.IMREAD_COLOR) # 이미지가 아니라 다른게 불러와지면 에러 발생
            preds = np.array(fa.get_landmarks(img)) # 검출 불가능하면 에러 발생
        except:
            print("periocular detect불가")
            print(dirs +'/'+ filelist[img_num])
            print("폴더명 :"+ str(folder_num))
            continue
            
            
        
        
#         print('filelist: %s' %filelist)
        

        x_max = 0
        x_min = len(img)
        y_max = 0
        y_min = len(img[0])
        
        try:
            for i in range(0, len(preds[0])):
                if (x_max < preds[0][i][0]):
                    x_max = preds[0][i][0]
                if (x_min > preds[0][i][0]):
                    x_min = preds[0][i][0]
                if (y_max < preds[0][i][1]):
                    y_max = preds[0][i][1]
                if (y_min > preds[0][i][1]):
                    y_min = preds[0][i][1]
        except: 
            print("periocular detect불가")
            print(dirs +'/'+ filelist[img_num])
            print("폴더명 :"+ str(folder_num))
            continue

         
        face_point= np.zeros((5,2))

        face_point[0][0] = preds[0][36][0] # point1 - right point of right eye
        face_point[0][1] = preds[0][36][1]
        face_point[1][0] = preds[0][39][0] # point2 - left point of right eye
        face_point[1][1] = preds[0][39][1] 
        face_point[2][0] = preds[0][42][0] # point3 - right point of left eye
        face_point[2][1] = preds[0][42][1]
        face_point[3][0] = preds[0][45][0] # point4 - left point of left eye
        face_point[3][1] = preds[0][45][1]
        face_point[4][0] = preds[0][30][0] # point5 - point of nose
        face_point[4][1] = preds[0][30][1]
        
        eye_y = int(0) # point 1, 2, 3, 4의 y좌표 평균(eye line)
        for i in range(0, 4):
            eye_y = eye_y + face_point[i][1]
        eye_y = eye_y/4.0
        
        roi_image = img.copy() 
        roi_image = img[int(y_min):int(face_point[4][1]+1), int(x_min):int(x_max+1)]
        
        try:
            resized = cv2.resize(roi_image, dsize=(91, 64), interpolation=cv2.INTER_AREA)
        except: 
            print("periocular detect불가")
            print(dirs +'/'+ filelist[img_num])
            print("폴더명 :"+ str(folder_num))
            
            continue
        
        if (os.path.isdir('./output')): # output폴더 있으면
            out_folder = './output/'+ str(folder_num)+'/'
            
        if(os.path.isdir(out_folder)): #output/person 에 save하기
            save_fname = str(folder_num) + '_' + str(img_num) + '_resized.jpg'
            cv2.imwrite('./output/'+ str(folder_num) + '/' + save_fname, resized)
            #print('./output/'+ str(folder_num) + '/' + save_fname)
        else: # output/person 이 없으면 만들고 save
            os.mkdir(out_folder)
            save_fname = str(folder_num) + '_' + str(img_num) + '_resized.jpg'
            cv2.imwrite('./output/'+ str(folder_num) + '/' + save_fname, resized)
            #print('./output/'+ str(folder_num) + '/' + save_fname)


periocular detect불가
./mask dataset/19/f_19_bott.jpg
폴더명 :18
periocular detect불가
./mask dataset/19/f_19_fron.jpg
폴더명 :18
periocular detect불가
./mask dataset/19/f_19_topp.jpg
폴더명 :18
periocular detect불가
./mask dataset/19/h_19_bott.jpg
폴더명 :18
periocular detect불가
./mask dataset/31/f_31_bott.jpg
폴더명 :30
periocular detect불가
./mask dataset/31/f_31_fron.jpg
폴더명 :30
periocular detect불가
./mask dataset/31/f_31_topp.jpg
폴더명 :30
periocular detect불가
./mask dataset/31/h_31_bott.jpg
폴더명 :30
periocular detect불가
./mask dataset/31/h_31_fron.jpg
폴더명 :30
periocular detect불가
./mask dataset/31/h_31_topp.jpg
폴더명 :30
periocular detect불가
./mask dataset/32/f_32_bott.jpg
폴더명 :31
periocular detect불가
./mask dataset/33/f_33_bott.jpg
폴더명 :32
periocular detect불가
./mask dataset/33/f_33_fron.jpg
폴더명 :32
periocular detect불가
./mask dataset/33/f_33_topp.jpg
폴더명 :32
periocular detect불가
./mask dataset/33/h_33_bott.jpg
폴더명 :32
periocular detect불가
./mask dataset/33/h_33_fron.jpg
폴더명 :32
periocular detect불가
./mask dataset/33/h_

위에서 print된 directory는 face alignment가 되지 않은 사진들임
이 사진들은 crop하여 저장하지 않았기 때문에 periocular 폴더에 사진이 한장만 들어있을수도있음 폴더확인해보기

데이터 총 1334명
